<h1>Agrupación y Segmentación de Vecindarios en la Ciudad de Toronto, canada </h1>

In [ ]:
Para esta tarea, se le pedirá que explore y agrupe los vecindarios en Toronto.

In [1]:
# Importando librerias necesarias
!pip install geopy
from geopy.geocoders import Nominatim # módulo para convertir una dirección en valores de latitud y longitud 

# librerías para mostrar imágenes 
from IPython.display import Image 
from IPython.core.display import HTML 
    
# librería para convertir un archivo json en un dataframe pandas
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # librería para graficar 

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.
